In [8]:
# -*- coding: utf-8 -*-
"""
Created on Fri Aug 16 17:14:42 2019
 
@author: mjofre
"""
###############################################################################
# Script to analyze and compute the Secure Key Rate of the continous variables.
# Whenever possible it will only be true for protocols with gaussian modulation.
###############################################################################
import numpy as np
import math
 
def TransmissionContinuousVariablesComputeSKR(SourcePulseRate,AttenuationdBforDistance,SystemDetectorPerformanceArray):
    # F. Laudenbach et al., “Continuous-Variable Quantum Key Distribution with Gaussian Modulation—The Theory of Practical Implementations,” Adv. Quantum Technol., vol. 1, no. 1, p. 1800011, 2018, doi: https://doi.org/10.1002/qute.201800011.
    # Supplementary material of G. Zhang et al., “An integrated silicon photonic chip platform for continuous-variable quantum key distribution,” Nat. Photonics, vol. 13, no. 12, Art. no. 12, Dec. 2019, doi: 10.1038/s41566-019-0504-5.
    RateAlice=SourcePulseRate
    Nlen=len(AttenuationdBforDistance)
    alphaLink=0.2 # 0.1 dB/Km link loss free-space attenuation#0.2dB/Km for 1550nm telecom fiber
    Distance=AttenuationdBforDistance/alphaLink*1000.0#np.linspace(0,350000,len(Measured_EMu))
    #Distance_plot=Distance*1e-3
    
    #Vmod=7.0 # modulation variance >1.0. It is important to optimize this value. Experimental study on discretely modulated continuous-variable quantum key distribution
    #ExcessNoiseAlice=0.01 # >0.0
    #ExcessNoiseDetector=0.01 # 0.0
    #Vn=1.5 # Variance noise electronics detection >1.0

    Vmod=SystemDetectorPerformanceArray[0]
    transmittanceBob=SystemDetectorPerformanceArray[1]
    attDetectors=SystemDetectorPerformanceArray[2]
    TimeJitterStd=SystemDetectorPerformanceArray[3]
    
    attLink=np.power(10,(-(alphaLink/10.0)*(Distance/1000.0))) # Channel transmittance
    attBob=attDetectors*transmittanceBob # denote the transmittance in Bob's side: internal transmittance of optical components and detector efficiency
    att=attLink*attBob#*(1-1/50); # Overall transmission and detection efficiency between Alice and Bob
    TransChannel=attLink # quantum channel transmission
        
    Trans=TransChannel*attBob

    MeanPhotonNumber=Vmod/2.0
    MuHomoDyne=1.0
    MuHeteroDyne=2.0
    hPlanck=6.62607004e-34

    RINsig=SystemDetectorPerformanceArray[3]#8e-11
    Bele=SystemDetectorPerformanceArray[4]#RateAlice
    RINlo=SystemDetectorPerformanceArray[5]#1.4e-7
    VRINLO=SystemDetectorPerformanceArray[6]#2.0
    g=SystemDetectorPerformanceArray[7]#20000#
    rho=SystemDetectorPerformanceArray[8]#0.85
    deltaUDAC=SystemDetectorPerformanceArray[9]#0.01    
    Upi=SystemDetectorPerformanceArray[10]#1.0
    UDAC=SystemDetectorPerformanceArray[11]#1.0
    psiPT=1.01-1.0# Fixed to a specific low value of 1%. SystemDetectorPerformanceArray[12]-1.0#1.2-1.0
    Npt=SystemDetectorPerformanceArray[13]#1.0
    meanPhotNumPT=SystemDetectorPerformanceArray[14]#600.0
    DelataLambda=SystemDetectorPerformanceArray[15]#8e-12 #[m]
    Nram=np.power(10.0,(SystemDetectorPerformanceArray[16]-30)/10.0)#np.power(10.0,(-75.0-30)/10.0)
    tau=SystemDetectorPerformanceArray[17]#1.0/RateAlice
    OpticalFreq=SystemDetectorPerformanceArray[18]#193.4e12 # [Hz]
    CMRR=SystemDetectorPerformanceArray[19]#1000.0
    Plo=SystemDetectorPerformanceArray[20]#8e-3
    NEP=SystemDetectorPerformanceArray[21]#4.5e-12
    nbits=SystemDetectorPerformanceArray[22]#10.0
    Ru=SystemDetectorPerformanceArray[23]#1.0
    VADCintr=SystemDetectorPerformanceArray[24]#1e-8
    gADC=SystemDetectorPerformanceArray[25]

    psiRIN=Trans*Vmod*np.sqrt(RINsig*Bele) # laser noise
    psiRinlo=(1.0/4.0)*RINlo*Bele*VRINLO # local oscillator noise
    psiMOD=Trans*Vmod*(np.pi*g*(deltaUDAC/Upi)+(1.0/2.0)*(np.pi*g*deltaUDAC/Upi)**2)**2# modulation noise
    #psiMODqpsk=Trans*Vmod*(np.pi*(deltaUDAC/UDAC)+(np.pi**2/2.0)*(deltaUDAC/UDAC)**2)**2# modulation noise
    psiPR=0.5*Vmod*((1.0+psiPT)/(Npt*meanPhotNumPT))# phase-recovery noise
    psiRaman=2.0*((DelataLambda*Nram*tau)/(hPlanck*OpticalFreq))*1e6# raman noise
    psiCMRRHomodyne=(MuHomoDyne/(4.0*CMRR**2))*(((hPlanck*OpticalFreq*(Vmod**2))/(4.0*tau*Plo))*RINsig*Bele+(tau**2/(hPlanck*OpticalFreq))*Plo*RINlo*Bele)# Common-Mode Rejection ratio#(MuHomoDyne/(4.0*CMRR**2))*(((hPlanck*OpticalFreq*(Vmod**2))/(4*tau*Plo))*RINsig*Bele+(tau/(hPlanck*OpticalFreq))*Plo*RINlo*Bele)# Common-Mode Rejection ratio
    psiCMRRHeterodyne=MuHeteroDyne*psiCMRRHomodyne
    psiDetHomodyne=MuHomoDyne*((NEP**2)*Bele*tau/(hPlanck*OpticalFreq*Plo))# Detection noise
    psiDetHeterodyne=MuHeteroDyne*psiDetHomodyne
    psiADCHomodyne=MuHomoDyne*((tau)/(hPlanck*OpticalFreq*(gADC**2)*(rho**2)*Plo))*((1.0/12.0)*((Ru**2)/np.power(2,2*nbits))+VADCintr)# ADC quantization noise
    psiADCHeterodyne=MuHeteroDyne*psiADCHomodyne

    ExcessNoiseAlice=psiRaman+psiMOD+psiPR
    ###########################################################################
    # Quantum key distribution over 25 km  with an all-fiber continuous-variable system Jérôme Lodewyck, Matthieu Bloch, Raúl García-Patrón, Simon Fossier, Evgueni Karpov, Eleni Diamanti, Thierry Debuisschert, Nicolas J. Cerf, Rosa Tualle-Brouri, Steven W. McLaughlin, and Philippe Grangier Phys. Rev. A 76, 042305 – Published 3 October 2007
    # Holevo-collective attacks
    # Continuous-Variable Quantum Key Distribution with Gaussian Modulation—The Theory of Practical Implementations
    # An integrated silicon photonic chip platform for continuous-variable quantum key distribution. Supplementary material
    
    ExcessNoiseDetectorHomodyne=MuHomoDyne*0.0
    ExcessNoiseDetectorHeterodyne=MuHeteroDyne*ExcessNoiseDetectorHomodyne

    VarianceV=Vmod+1    
    
    ChannelNoise=(1.0-TransChannel)/TransChannel+ExcessNoiseAlice/Trans # Ok
    DetectorNoise=(1.0-attBob)+ExcessNoiseDetectorHomodyne # Ok
    #One detector
    MuHomoDyne=1.0
    MuHeteroDyne=2.0
    ExcessNoiseAtAliceHomoDyne=ExcessNoiseAlice+ExcessNoiseDetectorHomodyne/Trans
    ExcessNoiseAtAliceHeteroDyne=ExcessNoiseAlice+ExcessNoiseDetectorHeterodyne/Trans
    ExcessNoiseAtBobHomoDyne=ExcessNoiseAlice+ExcessNoiseDetectorHomodyne#Trans*ExcessNoiseAtAliceHomoDyne/MuHomoDyne
    ExcessNoiseAtBobHeteroDyne=ExcessNoiseAlice+ExcessNoiseDetectorHeterodyne#Trans*ExcessNoiseAtAliceHeteroDyne/MuHeteroDyne
    #VarianceBob=Trans*Vmod+1.0+ExcessNoiseAtBob
    fsym=0.95 #error correction factor
    
    # Continuous-Variable Quantum Key Distribution with Gaussian Modulation—The Theory of Practical Implementations
    MutualInformationAliceBobHomoDyne=(MuHomoDyne/2.0)*np.log2(1.0+(Trans*Vmod/MuHomoDyne)/(1.0+ExcessNoiseAtBobHomoDyne))    
    MutualInformationAliceBobHeteroDyne=(MuHeteroDyne/2.0)*np.log2(1.0+(Trans*Vmod/MuHeteroDyne)/(1.0+ExcessNoiseAtBobHeteroDyne))
 
    TotalNoiseHomodyne=ChannelNoise+MuHomoDyne*DetectorNoise/Trans    # Ok
    TotalNoiseHeterodyne=ChannelNoise+MuHeteroDyne*DetectorNoise/Trans # Ok

    # An integrated silicon photonic chip platform for continuous-variable quantum key distribution. Supplementary material
    #MutualInformationAliceBobHomoDyne=(MuHomoDyne/2.0)*np.log2((attBob*Trans*(VarianceV+TotalNoiseHomodyne))/(attBob*(1.0/(Trans*(1.0/VarianceV+ChannelNoise))+MuHomoDyne*DetectorNoise)))
    #MutualInformationAliceBobHeteroDyne=(MuHeteroDyne/2.0)*np.log2((attBob*Trans*(VarianceV+TotalNoiseHeterodyne))/(attBob*(1.0/(Trans*(1.0/VarianceV+ChannelNoise))+MuHeteroDyne*DetectorNoise)))

    def SymplecticEigenValues12(a,b,c):
      lambda1=0.5*(np.sqrt((a+b)**2-4*c**2)+(b-a))
      lambda2=0.5*(np.sqrt((a+b)**2-4*c**2)-(b-a))
      return lambda1,lambda2

    def SymplecticEigenValues34(alpha,beta):
      lambda3=np.sqrt(0.5*(alpha+np.sqrt(alpha**2-4*beta)))
      lambda4=np.sqrt(0.5*(alpha-np.sqrt(alpha**2-4*beta)))
      return lambda3,lambda4

    def ValuesG(vAux):
      Gvalues=np.zeros((len(vAux)),dtype=np.float32)
      if (np.sum(vAux<=0.0)>0):
        Gvalues[vAux<=0.0]=0.0
        Gvalues[vAux>0.0]=(vAux[vAux>0.0]+1.0)*np.log2(vAux[vAux>0.0]+1.0)-(vAux[vAux>0.0])*np.log2(vAux[vAux>0.0])
      else:
        Gvalues=(vAux+1.0)*np.log2(vAux+1.0)-(vAux)*np.log2(vAux)
      return Gvalues
    
    # Homodyne
    aHomodyne=VarianceV
    bHomodyne=Trans*(VarianceV-1.0)+1.0+ExcessNoiseAtBobHomoDyne
    cHomodyne=np.sqrt(Trans*(VarianceV**2-1.0))
    
    alphaHomodyne=(aHomodyne*(aHomodyne*bHomodyne-cHomodyne**2)+bHomodyne+(aHomodyne**2+bHomodyne**2-2*cHomodyne**2)*MuHomoDyne*DetectorNoise)/(bHomodyne+MuHomoDyne*DetectorNoise)
    betaHomodyne=(aHomodyne*(aHomodyne*bHomodyne-cHomodyne**2)+((aHomodyne*bHomodyne-cHomodyne**2)**2)*MuHomoDyne*DetectorNoise)/(bHomodyne+MuHomoDyne*DetectorNoise)

    [lambda1Homodyne,lambda2Homodyne]=SymplecticEigenValues12(aHomodyne,bHomodyne,cHomodyne)
    [lambda3Homodyne,lambda4Homodyne]=SymplecticEigenValues34(alphaHomodyne,betaHomodyne)

    # Heterodyne
    aHeterodyne=VarianceV
    bHeterodyne=Trans*(VarianceV-1.0)+1.0+ExcessNoiseAtBobHeteroDyne
    cHeterodyne=np.sqrt(Trans*(VarianceV**2-1.0))

    alphaHeterodyne=(aHeterodyne*(aHeterodyne*bHeterodyne-cHeterodyne**2)+bHeterodyne+(aHeterodyne**2+bHeterodyne**2-2*cHeterodyne**2)*MuHeteroDyne*DetectorNoise)/(bHeterodyne+MuHeteroDyne*DetectorNoise)
    betaHeterodyne=(aHeterodyne*(aHeterodyne*bHeterodyne-cHeterodyne**2)+((aHeterodyne*bHeterodyne-cHeterodyne**2)**2)*MuHeteroDyne*DetectorNoise)/(bHeterodyne+MuHeteroDyne*DetectorNoise)
    
    [lambda1Heterodyne,lambda2Heterodyne]=SymplecticEigenValues12(aHeterodyne,bHeterodyne,cHeterodyne)
    [lambda3Heterodyne,lambda4Heterodyne]=SymplecticEigenValues34(alphaHeterodyne,betaHeterodyne)    
  
    # Collective attack
    HolevoInformationHomodyne=ValuesG((lambda1Homodyne-1.0)/2.0)+ValuesG((lambda2Homodyne-1.0)/2.0)-ValuesG((lambda3Homodyne-1.0)/2.0)-ValuesG((lambda4Homodyne-1.0)/2.0)
    HolevoInformationHeterodyne=ValuesG((lambda1Heterodyne-1.0)/2.0)+ValuesG((lambda2Heterodyne-1.0)/2.0)-ValuesG((lambda3Heterodyne-1.0)/2.0)-ValuesG((lambda4Heterodyne-1.0)/2.0)
    ####
    FER=0.0 #frame error rate
    vFractionSymbolsDisclosed=0.0 # the fraction of  bits  that  cannot  be  used  to  distil  the final key
    BetaEfficiencyReconciliation=1.0 # the reconciliation  efficiency

    HomodyneKeyBitRateLowerBond=RateAlice*fsym*(1.0-FER)*(1-vFractionSymbolsDisclosed)*(BetaEfficiencyReconciliation*MutualInformationAliceBobHomoDyne-HolevoInformationHomodyne)
    HeterodyneKeyBitRateLowerBond=RateAlice*fsym*(1.0-FER)*(1-vFractionSymbolsDisclosed)*(BetaEfficiencyReconciliation*MutualInformationAliceBobHeteroDyne-HolevoInformationHeterodyne)

    # Assuming similarity with QPSK or QAM
    # From John G. Proakis, "Digital Communications", Electrical Engineering Series, McGraw-Hill, Fourth Edition (2001). 
    PsignalPower=1.0
    PulseDuration=tau
    Er=PsignalPower*PulseDuration
    N0=Trans*1.0e-9 # It can be better computed following: https://www.nature.com/articles/srep19201
    RinfoHomoDyne=BetaEfficiencyReconciliation*(MuHomoDyne/2.0)*np.log2(1.0+2.0*Er/N0)
    RinfoHeteroDyne=BetaEfficiencyReconciliation*(MuHeteroDyne/2.0)*np.log2(1.0+2.0*Er/N0)
 
    SNRHomoDyne=((1.0/MuHomoDyne)*Trans*Vmod)/(1.0+(1.0/MuHomoDyne)*ExcessNoiseAtBobHomoDyne)
    QBERHomodyne=np.zeros_like(SNRHomoDyne)
    for iIterAuxErfc in range(0,len(SNRHomoDyne),1):
      #QBERHomodyne[iIterAuxErfc]=0.5*math.erfc(np.sqrt(SNRHomoDyne[iIterAuxErfc]/(2.0*RinfoHomoDyne[iIterAuxErfc])))
      QBERHomodyne[iIterAuxErfc]=0.5*math.erfc(np.sqrt(2.0*SNRHomoDyne[iIterAuxErfc])*np.sin(np.pi/4.0))
 
    SNRHeteroDyne=((1.0/MuHeteroDyne)*Trans*Vmod)/(1.0+(1.0/MuHeteroDyne)*ExcessNoiseAtBobHeteroDyne)
    QBERHeterodyne=np.zeros_like(SNRHeteroDyne)
    for iIterAuxErfc in range(0,len(SNRHeteroDyne),1):
      #QBERHeterodyne[iIterAuxErfc]=0.5*math.erfc(np.sqrt(SNRHeteroDyne[iIterAuxErfc]/(2.0*RinfoHeteroDyne[iIterAuxErfc])))
      QBERHeterodyne[iIterAuxErfc]=0.5*math.erfc(np.sqrt(2.0*SNRHeteroDyne[iIterAuxErfc])*np.sin(np.pi/4.0))

    return attLink,HomodyneKeyBitRateLowerBond,HeterodyneKeyBitRateLowerBond,TotalNoiseHomodyne,TotalNoiseHeterodyne,QBERHomodyne,QBERHeterodyne